In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import descartes
import math
import matplotlib.colors as mcolors
import pyproj
import folium

In [2]:
wdir = "../Dades/AMS_Observacions/"
crs_latlon = 'EPSG:4326'  # WGS84
crs_utm = "EPSG:32631"

In [3]:
df_no2 = pd.read_csv('../Dades/AMS_Observacions/gene_sconcno2_2023_xvpca_emep_port.csv') 
df_estacions = pd.read_csv('../Dades/AMS_Observacions/XVPCA_info_sconcno2_2023.csv')


# Convert 'Date' column to datetime format
df_no2['Date'] = pd.to_datetime(df_no2['Date'])

# Pivot the dataframe to have each station ID as a column and the Date as the index
# columnas_es = [col for col in df_no2.columns if col.startswith('ES')]
columnas_es =df_no2.drop(['Date'], axis=1)


# Transpose the dataframe to have each station ID as a row and the Date as the column
df_no2_transposed = df_no2.set_index('Date').transpose().reset_index()

# Melt the dataframe to have a 'Value' column for each hour
df_no2_melted = pd.melt(df_no2_transposed, id_vars=['index'], var_name='Date', value_name='no2')

# Rename the 'index' column to 'Station'
df_no2_melted.rename(columns={'index': 'Station'}, inplace=True)

# Drop rows with NaN values
df_no2_melted.dropna(subset=['no2'], inplace=True)

print(df_no2_melted)

# Join the melted dataframe with the station information dataframe
df_obs = pd.merge(df_no2_melted, df_estacions, left_on='Station', right_on='code', how='inner')

# Drop the 'code' column as it is redundant
df_obs.drop(columns=['code','type'], inplace=True)
df_obs['geometry'] = gpd.points_from_xy(df_obs['lon'], df_obs['lat'], crs=crs_utm)
print(df_obs)

        Station                 Date   no2
2       ES1480A  2022-12-31 23:00:00  62.0
3       ES1396A  2022-12-31 23:00:00  60.0
4       ES1992A  2022-12-31 23:00:00  45.0
5       ES0691A  2022-12-31 23:00:00  43.0
6       ES0692A  2022-12-31 23:00:00  45.0
...         ...                  ...   ...
560633  ES2017A  2023-12-31 22:00:00   7.0
560634  ES1930A  2023-12-31 22:00:00   1.0
560635  ES1948A  2023-12-31 22:00:00   3.0
560636  ES1855A  2023-12-31 22:00:00   2.0
560637  ES1854A  2023-12-31 22:00:00   1.0

[529219 rows x 3 columns]
        Station                 Date   no2        lat       lon  \
0       ES1480A  2022-12-31 23:00:00  62.0  41.398762  2.153472   
1       ES1396A  2022-12-31 23:00:00  60.0  41.378803  2.133098   
2       ES1992A  2022-12-31 23:00:00  45.0  41.387273  2.115661   
3       ES0691A  2022-12-31 23:00:00  43.0  41.403716  2.204736   
4       ES0692A  2022-12-31 23:00:00  45.0  41.370760  2.114771   
...         ...                  ...   ...        ...  

In [4]:
df_caliope = pd.read_csv('./NO2_no_h.csv') 

df_caliope.rename(columns={'sconcno2': 'no2', 'time': 'Date'}, inplace=True)

df_caliope['geometry'] = gpd.points_from_xy(df_caliope['lon'], df_caliope['lat'], crs=crs_utm)

df_combined = pd.concat([df_obs, df_caliope], ignore_index=True)

# Convert df_combined to a GeoDataFrame
df_combined = gpd.GeoDataFrame(df_combined, geometry='geometry', crs=crs_utm)

# Display the first few rows to verify the conversion
df_combined.head()


,Station,Date,no2,lat,lon,geometry
0,ES1480A,2022-12-31 23:00:00,62.0,41.398762,2.153472,POINT (2.153 41.399)
1,ES1396A,2022-12-31 23:00:00,60.0,41.378803,2.133098,POINT (2.133 41.379)
2,ES1992A,2022-12-31 23:00:00,45.0,41.387273,2.115661,POINT (2.116 41.387)
3,ES0691A,2022-12-31 23:00:00,43.0,41.403716,2.204736,POINT (2.205 41.404)
4,ES0692A,2022-12-31 23:00:00,45.0,41.370760,2.114771,POINT (2.115 41.371)


In [5]:
# Agrupar por 'geometry' y calcular la media de 'no2'
df_combined_mean = df_caliope.groupby('geometry')['no2'].mean().reset_index()

# Mostrar el resultado
print("Max NO2:", df_combined_mean['no2'].max())
print("Min NO2:", df_combined_mean['no2'].min())

Max NO2: 0.019521529590625
Min NO2: 6.8170541609375e-05


In [6]:
from scipy.spatial import cKDTree
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

stations_gdf = gpd.GeoDataFrame(df_obs.drop_duplicates(subset=['geometry']).copy(), geometry='geometry')
# Crear GeoDataFrame con puntos de predicción
obs_locations = gpd.GeoDataFrame(
    {'geometry': [Point(2.009802, 41.39216), Point(1.191975, 41.11588), 
                  Point(2.237875, 41.44398), Point(2.082141, 41.32177)]},
    crs=crs_utm
)

# Extraer coordenadas y valores del dataset conocido
coords = np.array([(geom.x, geom.y) for geom in df_combined_mean.geometry])
values = df_combined_mean['no2'].values

# Crear el árbol KD
tree = cKDTree(coords)

# Función IDW
def idw(x, y, tree, coords, values, power=2):
    """
    Interpolación por inverso de la distancia (IDW).
    - x, y: Coordenadas de predicción
    - tree: KDTree construido con los datos conocidos
    - coords: Coordenadas conocidas
    - values: Valores conocidos
    - power: Peso de la distancia
    """
    # Encontrar distancias y vecinos
    distances, idx = tree.query(np.c_[x, y], k=len(coords))
    
    # Manejo de distancias 0 (puntos coincidentes)
    distances[distances == 0] = 1e-10  # Evitar división por cero
    
    # Calcular pesos inversos a las distancias
    weights = 1 / distances**power
    
    # Normalizar los pesos
    weights /= np.sum(weights, axis=1, keepdims=True)
    
    # Calcular valores interpolados
    interpolated_values = np.sum(weights * values[idx], axis=1)
    return interpolated_values

# Coordenadas de los puntos a predecir
prediction_coords = np.array([(geom.x, geom.y) for geom in obs_locations.geometry])
x_pred, y_pred = prediction_coords[:, 0], prediction_coords[:, 1]

# Aplicar IDW a los puntos
obs_locations['predicted_value'] = idw(x_pred, y_pred, tree, coords, values)

print(obs_locations)
# Coordenadas de los puntos de las estaciones
station_coords = np.array([(geom.x, geom.y) for geom in stations_gdf.geometry])
x_station, y_station = station_coords[:, 0], station_coords[:, 1]

# Aplicar IDW a los puntos de las estaciones
stations_gdf['predicted_value'] = idw(x_station, y_station, tree, coords, values)

print(stations_gdf)
print(stations_gdf.head())


               geometry  predicted_value
0   POINT (2.01 41.392)         0.002725
1  POINT (1.192 41.116)         0.004979
2  POINT (2.238 41.444)         0.004959
3  POINT (2.082 41.322)         0.003409
       Station                 Date   no2        lat       lon  \
0      ES1480A  2022-12-31 23:00:00  62.0  41.398762  2.153472   
1      ES1396A  2022-12-31 23:00:00  60.0  41.378803  2.133098   
2      ES1992A  2022-12-31 23:00:00  45.0  41.387273  2.115661   
3      ES0691A  2022-12-31 23:00:00  43.0  41.403716  2.204736   
4      ES0692A  2022-12-31 23:00:00  45.0  41.370760  2.114771   
...        ...                  ...   ...        ...       ...   
2203   ES1348A  2023-01-02 12:00:00  16.0  42.368390  1.776814   
3579   ES1438A  2023-01-03 12:00:00  35.0  41.385366  2.154030   
12183  ES1815A  2023-01-09 10:00:00   3.0  41.346823  1.686575   
13418  ES1854A  2023-01-10 06:00:00   1.0  41.008212  0.831085   
86441  ES1642A  2023-02-28 23:00:00   6.0  41.935010  2.239901   

  

In [7]:
# Cargar el shapefile
archivo_shapefile = "../Dades/OpenTransportMap/Barcelona/roadlinks_ES511.shp"
gdf_barcelona = gpd.read_file(archivo_shapefile)
gdf_girona = gpd.read_file("../Dades/OpenTransportMap/Girona/roadlinks_ES512.shp")
gdf_lleida = gpd.read_file("../Dades/OpenTransportMap/Lleida/roadlinks_ES513.shp")
gdf_tarragona = gpd.read_file("../Dades/OpenTransportMap/Tarragona/roadlinks_ES514.shp")

# Concatenate the GeoDataFrames
gdf = pd.concat([gdf_barcelona, gdf_girona, gdf_lleida, gdf_tarragona], ignore_index=True)
# Mostrar las primeras filas de la tabla de atributos
#print(gdf['functional'].head())

# Obtener los tipos únicos de la columna 'functional'
tipos_functional = gdf['functional'].unique()

# Asignar valores de prioridad a las categorías
valores_prioridad = {
    'mainRoad': 4 / 4,
    'firstClass': 3 / 4,
    'secondClass': 2 / 4,
    'thirdClass': 1 / 4,
    'fourthClass': 0,
    'fifthClass': 0,    
}

# Crear una nueva columna en el GeoDataFrame con los valores de prioridad
gdf['prioridad'] = gdf['functional'].map(valores_prioridad)

# Crear un nuevo DataFrame con las columnas 'sourceid', 'functional' y 'prioridad'
df_transport = gdf[['sourceid', 'prioridad','geometry']].drop_duplicates().reset_index(drop=True)
# Eliminar filas con valores de prioridad igual a 0
df_transport = df_transport[df_transport['prioridad'] != 0]
print(df_transport.head())

# Guardar el GeoDataFrame en un archivo CSV
#gdf.to_csv("./carreteres.csv", index=False)

     sourceid  prioridad                                           geometry
8   247323996       0.25  LINESTRING (2.229 41.66569, 2.229 41.66606, 2....
10  199276193       0.25  LINESTRING (1.91614 41.5961, 1.91615 41.59607,...
11  199276720       1.00    LINESTRING (1.63758 41.73453, 1.63799 41.73411)
13  199316496       1.00  LINESTRING (1.92758 41.79895, 1.92656 41.7982,...
14  199333176       1.00  LINESTRING (2.16105 41.93252, 2.16175 41.93245...


In [8]:
from shapely.geometry import LineString
from shapely.ops import nearest_points

roads_gdf = gpd.GeoDataFrame(df_transport.copy(), geometry='geometry')

stations_gdf = stations_gdf[stations_gdf.geometry.is_valid]
roads_gdf = roads_gdf[roads_gdf.geometry.is_valid]

def encontrar_carretera_cercana(punto):
    # Busca los índices de las carreteras cercanas
    posibles_indices = list(roads_gdf.sindex.intersection(punto.buffer(0.01).bounds))
    
    # Si hay carreteras cercanas
    if posibles_indices:
        # Filtrar y encontrar la más cercana
        carreteras_cercanas = roads_gdf.iloc[posibles_indices]
        distancias = carreteras_cercanas.geometry.distance(punto)
        indice_mas_cercano = distancias.idxmin()
        road = roads_gdf.loc[indice_mas_cercano]
        return road['prioridad']
    
    return None

In [9]:
# Aplicar la función a cada punto
import warnings

# Aplicar la función a cada punto en stations_gdf
warnings.filterwarnings('ignore')

stations_gdf['carretera_cercana'] = stations_gdf['geometry'].apply(encontrar_carretera_cercana)

# Mostrar los resultados
print(stations_gdf[['geometry', 'carretera_cercana','no2']].head())

               geometry  carretera_cercana   no2
0  POINT (2.153 41.399)               0.75  62.0
1  POINT (2.133 41.379)               0.25  60.0
2  POINT (2.116 41.387)               0.25  45.0
3  POINT (2.205 41.404)               0.25  43.0
4  POINT (2.115 41.371)               0.25  45.0


In [12]:
def encontrar_road_dict(punto, carreteras_gdf, buffer_distance=0.01):
    # Crear un buffer alrededor del punto
    punto_buffer = punto.buffer(buffer_distance)
    
    # Encontrar carreteras dentro del buffer
    road_dict = carreteras_gdf[carreteras_gdf.intersects(punto_buffer)]
    
    # Si no hay carreteras cercanas, devolver None
    if road_dict.empty:
        return None
    
    # Calcular distancias y otra información
    road_dict['distancia'] = road_dict.geometry.apply(lambda geom: geom.distance(punto))
    
    # Ordenar por distancia
    road_dict = road_dict.sort_values('distancia')
    
    # Extraer información relevante
    # Eliminar duplicados basados en la distancia
    road_dict = road_dict.drop_duplicates(subset=['distancia']).head(10)
    
    info_carreteras = road_dict.apply(lambda row: {
        'sourceid': row['sourceid'],
        'prioridad': row['prioridad'],
        'distancia': row['distancia']
    }, axis=1).tolist()
    
    return info_carreteras

# Aplicar la función a cada punto
def procesar_estaciones(estaciones_gdf, buffer_size=0.05):
    # Crear una copia para no modificar el original
    estaciones_procesadas = estaciones_gdf.copy()
    
    # Aplicar la función de búsqueda de carreteras cercanas
    estaciones_procesadas['road_dict'] = estaciones_procesadas.geometry.apply(
        lambda punto: encontrar_road_dict(punto, roads_gdf, buffer_distance=buffer_size)
    )
    
    return estaciones_procesadas

# Ejecutar el procesamiento
stations_gdf = procesar_estaciones(stations_gdf)

print(stations_gdf.head())
print(stations_gdf['road_dict'].iloc[0])

   Station                 Date   no2        lat       lon  \
0  ES1480A  2022-12-31 23:00:00  62.0  41.398762  2.153472   
1  ES1396A  2022-12-31 23:00:00  60.0  41.378803  2.133098   
2  ES1992A  2022-12-31 23:00:00  45.0  41.387273  2.115661   
3  ES0691A  2022-12-31 23:00:00  43.0  41.403716  2.204736   
4  ES0692A  2022-12-31 23:00:00  45.0  41.370760  2.114771   

               geometry  predicted_value  carretera_cercana  \
0  POINT (2.153 41.399)         0.004002               0.75   
1  POINT (2.133 41.379)         0.005388               0.25   
2  POINT (2.116 41.387)         0.003676               0.25   
3  POINT (2.205 41.404)         0.004385               0.25   
4  POINT (2.115 41.371)         0.004230               0.25   

                                 carreteras_cercanas  \
0  [{'sourceid': 237659863, 'prioridad': 0.75, 'd...   
1  [{'sourceid': 239293914, 'prioridad': 0.25, 'd...   
2  [{'sourceid': 34038369, 'prioridad': 0.25, 'di...   
3  [{'sourceid': 2377870

In [20]:

df_station = stations_gdf[['geometry', 'predicted_value', 'road_dict', 'no2']]

df_station.head()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Preparar datos para el modelo
def extraer_caracteristicas(road_dict):
    if not road_dict or len(road_dict) == 0:
        return {
            'num_carreteras': 0,
            'prioridad_media': 0,
            'prioridad_min': 0,
            'prioridad_max': 0,
            'distancia_media': 0,
            'distancia_min': 0
        }
    
    # Extraer información de las carreteras
    num_carreteras = len(road_dict)
    prioridades = [road['prioridad'] for road in road_dict]
    distancias = [road['distancia'] for road in road_dict]
    
    return {
        'num_carreteras': num_carreteras,
        'prioridad_media': np.mean(prioridades),
        'prioridad_min': np.min(prioridades),
        'prioridad_max': np.max(prioridades),
        'distancia_media': np.mean(distancias),
        'distancia_min': np.min(distancias)
    }

# Preparar el DataFrame de características
def preparar_datos(df_station):
    # Extraer características de las carreteras
    
    caracteristicas_carreteras = df_station['road_dict'].apply(extraer_caracteristicas)
    
    # Convertir a DataFrame
    df_caracteristicas = pd.DataFrame(caracteristicas_carreteras.tolist())
    
    # Añadir la columna de NO2
    df_caracteristicas['no2'] = df_station['no2']
    df_caracteristicas['predicted_value'] = df_station['predicted_value']
    
    return df_caracteristicas

dades_forest = preparar_datos(df_station)

print(dades_forest.head())

   num_carreteras  prioridad_media  prioridad_min  prioridad_max  \
0              10             0.75           0.75           0.75   
1              10             0.35           0.25           0.75   
2              10             0.40           0.25           0.75   
3              10             0.25           0.25           0.25   
4              10             0.25           0.25           0.25   

   distancia_media  distancia_min   no2  predicted_value  
0         0.001016       0.000304  62.0         0.004002  
1         0.002787       0.002102  60.0         0.005388  
2         0.001980       0.001486  45.0         0.003676  
3         0.001888       0.001422  43.0         0.004385  
4         0.001177       0.000299  45.0         0.004230  


In [ ]:
features = ['num_carreteras', 'prioridad_media', 'prioridad_min', 'prioridad_max', 
            'distancia_media', 'distancia_min']
X_features = X[features]
y = X['no2']

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,  # Número de árboles
    random_state=42,
    max_depth=10,  # Profundidad máxima de los árboles
    min_samples_split=2,
    min_samples_leaf=1
)
rf_model.fit(X_train_scaled, y_train)

# Predecir y evaluar
y_pred = rf_model.predict(X_test_scaled)

# Métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R² Score:", r2)

# Importancia de características
importancia = pd.DataFrame({
    'caracteristica': features,
    'importancia': rf_model.feature_importances_
}).sort_values('importancia', ascending=False)
print("\nImportancia de características:")
print(importancia)

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

# Cargar y preparar los datos
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Cargar los datos

data = df_combined.sample(n=10000, random_state=42)
data.rename(columns={'Date': 'date', 'no2': 'concentration'}, inplace=True)
# Convert 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce')
data.dropna(subset=['date'], inplace=True)
# Generar características adicionales (temporales y geográficas)
data['hour'] = data['date'].dt.hour
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month

# Variables predictoras y objetivo
X = data[['concentration', 'lat', 'lon', 'hour', 'day', 'month']]
y = data['concentration'].shift(-1)  # El siguiente valor de concentración

# Eliminar valores nulos generados por el shift
X = X[:-1]
y = y[:-1]

# Entrenar el modelo con todos los datos
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Predicción con un nuevo conjunto de datos
# Sustituye estos valores por los datos de entrada para la predicción
new_data = pd.DataFrame({
    'concentration': [0.46611],  # Concentración inicial
    'lat': [41.39216],
    'lon': [2.009802],
    'hour': [0],
    'day': [1],
    'month': [1]
})
new_concentration = model.predict(new_data)
print(f"Nueva concentración: {new_concentration[0]}")



Nueva concentración: 0.6651062339418704


In [23]:
obs_loc = obs_locations.copy()
obs_loc['lat'] = obs_loc.geometry.y
obs_loc['lon'] = obs_loc.geometry.x
obs_loc = obs_loc.drop(columns=['geometry'])
obs_loc.rename(columns={'predicted_value': 'concentration'}, inplace=True)

# Generate a date range from January 1, 2023 to December 31, 2023, hourly
date_range = pd.date_range(start='2023-01-01 00:00:00', end='2023-12-31 23:00:00', freq='H')

# Repeat the date range for each point
obs_loc = obs_loc.loc[obs_loc.index.repeat(len(date_range))].reset_index(drop=True)

# Assign the repeated date range to the 'Date' column
obs_loc['date'] = pd.concat([pd.Series(date_range)] * 4, ignore_index=True)

# Add an ID column
obs_loc['id'] = obs_loc.index + 1

print(obs_loc.head())
# Fit the model for each row in obs_loc
obs_loc['predicted_concentration'] = obs_loc.apply(
    lambda row: model.predict(pd.DataFrame({
        'concentration': [row['concentration']],
        'lat': [row['lat']],
        'lon': [row['lon']],
        'hour': [row['date'].hour],
        'day': [row['date'].day],
        'month': [row['date'].month]
    }))[0], axis=1
)

print(obs_loc.head())

   concentration       lat       lon                date  id
0       0.465024  41.39216  2.009802 2023-01-01 00:00:00   1
1       0.465024  41.39216  2.009802 2023-01-01 01:00:00   2
2       0.465024  41.39216  2.009802 2023-01-01 02:00:00   3
3       0.465024  41.39216  2.009802 2023-01-01 03:00:00   4
4       0.465024  41.39216  2.009802 2023-01-01 04:00:00   5
   concentration       lat       lon                date  id  \
0       0.465024  41.39216  2.009802 2023-01-01 00:00:00   1   
1       0.465024  41.39216  2.009802 2023-01-01 01:00:00   2   
2       0.465024  41.39216  2.009802 2023-01-01 02:00:00   3   
3       0.465024  41.39216  2.009802 2023-01-01 03:00:00   4   
4       0.465024  41.39216  2.009802 2023-01-01 04:00:00   5   

   predicted_concentration  
0                 0.665106  
1                 0.665082  
2                 0.333217  
3                 0.333175  
4                 0.332464  


In [24]:
# Convertir la columna 'predicted_concentration' en 'concentration'
obs_loc['concentration'] = obs_loc['predicted_concentration']

# Eliminar la columna 'predicted_concentration'
obs_loc.drop(columns=['predicted_concentration'], inplace=True)

# Guardar el DataFrame resultante en un archivo CSV
obs_loc.to_csv('concentracio_temps2.csv', index=False)

print("Archivo CSV guardado como 'predicted_concentration.csv'")

Archivo CSV guardado como 'predicted_concentration.csv'


In [25]:
print(obs_loc)

       concentration       lat       lon                date     id
0           0.665106  41.39216  2.009802 2023-01-01 00:00:00      1
1           0.665082  41.39216  2.009802 2023-01-01 01:00:00      2
2           0.333217  41.39216  2.009802 2023-01-01 02:00:00      3
3           0.333175  41.39216  2.009802 2023-01-01 03:00:00      4
4           0.332464  41.39216  2.009802 2023-01-01 04:00:00      5
...              ...       ...       ...                 ...    ...
35035       1.592791  41.32177  2.082141 2023-12-31 19:00:00  35036
35036       1.592768  41.32177  2.082141 2023-12-31 20:00:00  35037
35037       1.592740  41.32177  2.082141 2023-12-31 21:00:00  35038
35038       1.592198  41.32177  2.082141 2023-12-31 22:00:00  35039
35039       1.012400  41.32177  2.082141 2023-12-31 23:00:00  35040

[35040 rows x 5 columns]


In [26]:
"""from multiprocessing import Pool

stations = gpd.GeoDataFrame(df_obs[['Station', 'geometry']].drop_duplicates().reset_index(drop=True), geometry='geometry')
stations['buffer'] = stations.geometry.buffer(100)

# Leer las carreteras desde el DataFrame
carreteras = df_transport

# Crear un índice espacial para las carreteras
carreteras_sindex = carreteras.sindex

def encontrar_tipo_carretera(buffer):
    # Encontrar las carreteras que intersectan con el buffer
    posibles_intersecciones = list(carreteras_sindex.intersection(buffer.bounds))
    carreteras_intersectadas = carreteras.iloc[posibles_intersecciones]
    
    # Filtrar las carreteras que realmente intersectan con el buffer
    intersecciones = carreteras_intersectadas[carreteras_intersectadas.intersects(buffer)]
    
    if not intersecciones.empty:
        intersecciones['buffer_id'] = buffer.name
        return intersecciones[['buffer_id', 'sourceid', 'prioridad']]
    else:
        return pd.DataFrame(columns=['buffer_id', 'sourceid', 'prioridad'])

# Procesar los buffers en paralelo
with Pool() as pool:
    resultados = pool.map(encontrar_tipo_carretera, stations['buffer'])

# Combinar los resultados
resultados_df = pd.concat(resultados, ignore_index=True)

# Mostrar resultados
print(resultados_df)"""


"from multiprocessing import Pool\n\nstations = gpd.GeoDataFrame(df_obs[['Station', 'geometry']].drop_duplicates().reset_index(drop=True), geometry='geometry')\nstations['buffer'] = stations.geometry.buffer(100)\n\n# Leer las carreteras desde el DataFrame\ncarreteras = df_transport\n\n# Crear un índice espacial para las carreteras\ncarreteras_sindex = carreteras.sindex\n\ndef encontrar_tipo_carretera(buffer):\n    # Encontrar las carreteras que intersectan con el buffer\n    posibles_intersecciones = list(carreteras_sindex.intersection(buffer.bounds))\n    carreteras_intersectadas = carreteras.iloc[posibles_intersecciones]\n    \n    # Filtrar las carreteras que realmente intersectan con el buffer\n    intersecciones = carreteras_intersectadas[carreteras_intersectadas.intersects(buffer)]\n    \n    if not intersecciones.empty:\n        intersecciones['buffer_id'] = buffer.name\n        return intersecciones[['buffer_id', 'sourceid', 'prioridad']]\n    else:\n        return pd.DataFrame